In [ ]:
# cuda + pytorch

In [ ]:
# 安装深度学习框架和d2l软件包
conda create --name d2l python=3.9 -y
conda activate d2l
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
pip install d2l==0.17.6

In [ ]:
import torch

# 当前安装的 PyTorch 库的版本
print(torch.__version__)
# 检查 CUDA 是否可用，即你的系统有 NVIDIA 的 GPU
print(torch.cuda.is_available())

2.6.0+cu124
True


In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.4605, 0.1687, 0.4247],
        [0.0367, 0.6319, 0.0521],
        [0.4452, 0.6081, 0.6473],
        [0.8934, 0.6002, 0.3884],
        [0.1990, 0.9620, 0.4015]])


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
device = 'cuda'
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312232  [   64/60000]
loss: 2.289656  [ 6464/60000]
loss: 2.274499  [12864/60000]
loss: 2.262028  [19264/60000]
loss: 2.247888  [25664/60000]
loss: 2.225684  [32064/60000]
loss: 2.226529  [38464/60000]
loss: 2.196468  [44864/60000]
loss: 2.194497  [51264/60000]
loss: 2.159559  [57664/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.152814 

Epoch 2
-------------------------------
loss: 2.172198  [   64/60000]
loss: 2.154941  [ 6464/60000]
loss: 2.102340  [12864/60000]
loss: 2.105972  [19264/60000]
loss: 2.063147  [25664/60000]
loss: 2.014380  [32064/60000]
loss: 2.026207  [38464/60000]
loss: 1.953954  [44864/60000]
loss: 1.957921  [51264/60000]
loss: 1.880415  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.877905 

Epoch 3
-------------------------------
loss: 1.926643  [   64/60000]
loss: 1.886234  [ 6464/60000]
loss: 1.773539  [12864/60000]
loss: 1.797223  [19264/60000]
loss: 1.700803  [25664/60000]
loss: 1.661333  [32064/600

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [15]:
torch.accelerator.current_accelerator()

device(type='cuda')